In [1]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 4521.89it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4534.48it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 3246044.84it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2118/2118 [00:00<00:00, 5614.50it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3375.30it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 122/122 [00:00<00:00, 3901.41it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1115/1115 [00:00<00:00, 3264.95it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 237/237 [00:00<00:00, 4494.81it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2300/2300 [00:00<00:00, 3319.89it/s]


In [20]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [21]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DSSM.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 300)          2893500     text_left[0][0]                  
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 300)          2893500     text_right[0][0]                 
_______________________________________________________________________________

In [25]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [28]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=4, batch_size=64, shuffle=True)
len(train_generator)

32

In [29]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/20
32/32 [==============================] - 5s 172ms/step - loss: 0.2200
Validation: normalized_discounted_cumulative_gain@3(0): 0.5059929954248539 - normalized_discounted_cumulative_gain@5(0): 0.5826753049711647 - mean_average_precision(0): 0.5333990450662669
Epoch 2/20
32/32 [==============================] - 5s 169ms/step - loss: 0.1937
Validation: normalized_discounted_cumulative_gain@3(0): 0.45644225063761773 - normalized_discounted_cumulative_gain@5(0): 0.5450868589210878 - mean_average_precision(0): 0.4982938490478807
Epoch 3/20
32/32 [==============================] - 5s 164ms/step - loss: 0.1780
Validation: normalized_discounted_cumulative_gain@3(0): 0.5003176257352125 - normalized_discounted_cumulative_gain@5(0): 0.5796050417476545 - mean_average_precision(0): 0.5299744448130818
Epoch 4/20
32/32 [==============================] - 6s 174ms/step - loss: 0.1677
Validation: normalized_discounted_cumulative_gain@3(0): 0.5209429726936254 - normalized_discounted_cumulative_g